# Define prediction problem and superfeatures for your dataset

This tutorial shows how to define the prediction problem (what to predict) and the superfeatures (groups of features that are usually missing jointly) for your dataset.

Note: Also define a datasplit

### Define paths 

In [14]:
# which dataset to work on 
dataset_name   = "synthetic_1"

In [15]:
# data specifications 
data_dir            = "../../../data/ts/" + dataset_name + "/fully_observed/" 
data_file          = data_dir + dataset_name + '_static.csv.gz' 
temporal_data_file = data_dir + dataset_name + '_ts_eav.csv.gz' 

# file to save problem
problem_file = data_dir + 'problem/' + 'problem.yaml'

# file to save superfeatures
superfeature_mapping_file = data_dir + 'superfeatures.csv'

# file for datasplit 
folds_file = data_dir + 'folds/' + 'fold_list.hkl'

### Imports

In [16]:
%load_ext autoreload
%autoreload 2

import sys
import os

# navigate to afa directory 
sys.path.insert(0, os.path.abspath('../../afa'))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Define problem

We define the problem, by setting what we want to predict. We save the problem in a .yaml file for faster loading.

In [17]:
from afa.data_modelling.problem.utils import load_problem_specs, save_problem_specs

In [18]:
# define problem specifications
problem_specs = { 'label_name' : ['Y_ts'], 
                  'problem'    : 'online',
                  'treatment' : None, 
                  'max_seq_len' : 10
                }

# save
save_problem_specs( problem_specs  = problem_specs , problem_file = problem_file )

In [19]:
problem_specs = load_problem_specs(problem_file = problem_file)
problem_specs['treatment']

## Define superfeature_mapping

Superfeatures contain multiple features that are usually acquired/ missing jointly. Think e.g. of an image where the image is the superfeature and the pixels are the features.   
They are thus especially important for defining the missingness process. 
If no superfeatures are defined, the default assumption is that every feature is also its own superfeature. 

Note: The superfeature generation for synthetic data is already included in the preparation00 tutorial. 

You can test the superfeature mapping by loading the data with the specified file in tutorial_classification_static.ipynb

### Option 1: Create superfeature mapping directly via a .csv file
Fill a .csv file by 
- listing superfeature names as columnnames
- writing the feature names below the corresponding superfeatures (can have different length columns). Make sure the feature names are spelled exactly how you load them in the dataframe 

### Option 2: Define them here and save the mapping 
A second option is to define them as a dictionary and save it. 

In [20]:
from afa.data_modelling.datasets.superfeatures.utils import save_name_mapping

In [21]:
superfeature2feature_name_mapping = \
    { 'superX0' : ['X0', 'X1'], 
      'superY'  : ['Y' ], 
      'superX0_ts' : ['X0_ts'], 
      'superX1_ts' : ['X1_ts'], 
      'superX2_ts' : ['X2_ts', 'X3_ts'] }

save_name_mapping( superfeature2feature_name_mapping , mapping_file  = superfeature_mapping_file   )  

### Option 3: Prepared superfeature mappings (not recommended)
Lastly, for specific datasets, the superfeature mapping creation can be stored automatically executed. 

In [22]:
from configurations.data_settings.define_data_settings_ts import generate_superfeature_mapping_ts
superfeature2feature_name_mapping = generate_superfeature_mapping_ts( dataset_name ,  data_dir = data_dir )

In [23]:
superfeature2feature_name_mapping

{'superX0': ['X0', 'X1'],
 'superY': ['Y'],
 'superX0_ts': ['X0_ts'],
 'superX1_ts': ['X1_ts'],
 'superX2_ts': ['X2_ts', 'X3_ts']}

## Test by loading superfeature mapping 

### Test 1: load superfeature mapping by itself 

In [24]:
from afa.data_modelling.datasets.superfeatures.utils import load_superfeature2feature_name_mapping
superfeature2feature_name_mapping = load_superfeature2feature_name_mapping( superfeature_mapping_file ) 

In [25]:
superfeature2feature_name_mapping

{'superX0': ['X0', 'X1'],
 'superY': ['Y'],
 'superX0_ts': ['X0_ts'],
 'superX1_ts': ['X1_ts'],
 'superX2_ts': ['X2_ts', 'X3_ts']}

## Test: load dataset with problem and superfeature mapping 

In [26]:
from afa.data_modelling.datasets.data_loader.data_loader_ts import DataLoader_ts

In [27]:
# load data
data_loader = DataLoader_ts( data_file                  = data_file,
                             temporal_data_file         = temporal_data_file,
                             superfeature_mapping_file  = superfeature_mapping_file,
                             problem_file               = problem_file)
dataset = data_loader.load() 

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 746.70it/s]


In [28]:
dataset.problem

'online'

In [29]:
# check superfeature mapping
dataset.superfeature2feature_name_mapping

{'superX0': ['X0', 'X1'],
 'superY': ['Y'],
 'superX0_ts': ['X0_ts'],
 'superX1_ts': ['X1_ts'],
 'superX2_ts': ['X2_ts', 'X3_ts']}

In [30]:
# check if resulting feature/superfeature names are correct 
dataset.feature_name

{'temporal': ['X0_ts', 'X1_ts', 'X2_ts', 'X3_ts'],
 'data': ['X0', 'X1', 'Y'],
 'treatment': None,
 'label': ['Y_ts'],
 'super_data': ['superX0', 'superY'],
 'super_temporal': ['superX0_ts', 'superX1_ts', 'superX2_ts']}

## Define data split 

In [31]:
# define the datasplit 
dataset.multi_split( prob_list = [0.8,0.1,0.1], split_names = ["train", "val", "test"])

# save the datasplot 
dataset.save_folds( data_dir ) 

In [32]:
# load data
data_loader = DataLoader_ts(   data_file                  = data_file,
                               temporal_data_file         = temporal_data_file,
                               superfeature_mapping_file  = superfeature_mapping_file,
                               problem_file               = problem_file, 
                               folds_file                 = folds_file)
dataset = data_loader.load() 

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 866.87it/s]


In [33]:
data = dataset.get_data(fold = 0, split = "train") 